## Project Summary 

Harmful algal blooms (HABs), particularly cyanobacterial blooms (CBs), are among the most critical environmental issues facing inland water bodies. These blooms pose significant threats to global public health and aquatic ecosystem stability. Chaohu Lake, the fifth-largest freshwater lake in China, faces severe eutrophication and frequent CBs due to agricultural runoff, industrial, and domestic pollution. 

Remote sensing technology enables real-time detection and long-term monitoring of CBs. This study focuses on Chaohu Lake as a case study, aiming to quantify and visualize the occurrence of CBs in the region over the past decade, providing insights into the impact of human activities on water quality and ecosystem health.

### Problem Statement 

This application addresses two key research questions: 

1. What is the spatial distribution of cyanobacterial blooms (CBs) in Chaohu Lake? 

2. What temporal patterns characterize the occurrence of CBs in Chaohu Lake? 

By leveraging remote sensing technology and indices, the application provides a cost-effective, real-time solution for quantifying and visualizing CB occurrences, enabling a deeper understanding of their spatial and temporal dynamics to inform management strategies.

### End User 

This application is designed to serve environmental researchers, policymakers, local authorities, and residents in the Chaohu Lake region.

For researchers, it provides real-time monitoring of cyanobacterial blooms (CBs), offering insights into their spatial distribution and temporal patterns, which are essential for scientific analysis and environmental management.

For policymakers and local authorities, the tool enables data-driven decision-making to implement effective mitigation strategies, allocate resources efficiently, and establish long-term action plans to address water quality challenges.

For residents, the application raises awareness about water quality issues and associated health risks, empowering communities to take measures and engage in sustainability efforts. 

### Data

This application uses Sentinel-2 MSI imagery from the Google Earth Engine platform. The image collection spans from 2018 to 2020 and is filtered for cloud coverage under 20%. Additionally, vegetation and water indices such as MNDWI, FAI, and TI are derived for analysis.

### Methodology

We use median composites and custom cloud masking for preprocessing. Water bodies are extracted using MNDWI, and bloom areas are identified by combining FAI and TI thresholds. We calculate bloom area statistics, classify bloom severity, and compute the Spatial Distribution Frequency Index (SDFI) for spatial analysis.

### Interface

This web application's interactive interface is tailored for researchers, policymakers, and residents of the Chaohu Lake region. Researchers can explore customizable map layers and time-series charts to analyze bloom severity and spatiotemporal trends. Policymakers benefit from instant data export, clear overlays, and intuitive legends, making evidence-based decisions easier. Residents access user-friendly controls, legends, and point queries to understand local bloom conditions and raise awareness. Together, these interactive features ensure that all users—from experts to the general public—can easily extract actionable insights, connecting satellite data with real-world environmental management.

## The Application 

:::{.column-page}

<iframe src='https://sjlin24.users.earthengine.app/view/hexageos24ucl' width='100%' height='700px'></iframe>

:::
## How it Works 

### 3.1 Data Processing

In this part, the code below defines Chaohu Lake's rectangular study area, centers the map on it, filters Sentinel-2 images within a specific date range and cloud cover threshold, and creates a composite image for analysis.

```js
// define study area - Chaohu Lake
var chaohu = ee.Geometry.Polygon(
  [[[117.220, 31.799],
    [117.220, 31.387],
    [117.937, 31.387],
    [117.937, 31.799]]]);


// set the center
Map.centerObject(chaohu, 10);

////////////////import data////////////////

// set time range
var startDate = '2018-05-01';
var endDate = '2020-11-01';

// import Sentinel-2 data and filter
var s2 = ee.ImageCollection('COPERNICUS/S2')
  .filterBounds(chaohu)
  .filterDate(startDate, endDate)
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
  
var compositeImage = s2.mosaic(); //Synthesize images within the same time period
```
### 3.2 Remote Sensing Indices
To effectively identify cyanobacterial blooms, we calculate and apply three key remote sensing indices:
1. Modified Normalized Difference Water Index (MNDWI)
Used to distinguish water bodies from land. It is calculated as:
$$
\text{MNDWI} = \frac{\rho_{\text{green}} - \rho_{\text{swir}}}{\rho_{\text{green}} + \rho_{\text{swir}}}
$$
where $\rho_{\text{Green}}$ represents the reflectance of the green band (B3), and $\rho_{\text{SWIR}}$ epresents the reflectance of the shortwave infrared band (B11).
```js
// Calculate MNDWI
var mndwi = medianComposite.normalizedDifference(['B3', 'B11']).rename('MNDWI');
```

2. Floating Algae Index (FAI)
Used to enhance the detection of floating algae by minimizing atmospheric effects. It is calculated as:
$$
\text{FAI} = \rho_{\text{nir}} - \rho_{\text{nir}}'
$$
where $\rho'_{\text{NIR}}$ is the baseline reflectance of the near-infrared band, calculated as:
$$
\rho_{\text{nir}}' = \rho_{\text{red}} + (\rho_{\text{swir}} - \rho_{\text{red}}) \times \frac{\lambda_{\text{nir}} - \lambda_{\text{red}}}{\lambda_{\text{swir}} - \lambda_{\text{red}}}
$$

```js
// Calculate FAI
var nirPrime = red.add(
  swir.subtract(red).multiply((833 - 664.5) / (1613.7 - 664.5))
);
var fai = nir.subtract(nirPrime).rename('FAI');
```
3. Turbidity Index (TI)
The TI is used to eliminate the influence of highly turbid water bodies on the detection of cyanobacterial blooms. It is calculated as:
$$
TI = \rho_{red} - \rho_{green} - \left( \frac{\rho_{nir} - \rho_{green}}{2} \right)
$$
```js
// Calculate TI
var ti = red.subtract(green).subtract(
  nir.subtract(green.multiply(0.5))
).rename('TI');
```
Complete Index Calculation Function:
```js
function calculateIndices(image) {
  // Normalize reflectance
  var green = image.select('B3').divide(10000);
  var red = image.select('B4').divide(10000);
  var nir = image.select('B8').divide(10000);
  var swir = image.select('B11').divide(10000);
  
  // Calculate FAI
  var nirPrime = red.add(swir.subtract(red).multiply((833 - 664.5) / (1613.7 - 664.5)));
  var fai = nir.subtract(nirPrime).rename('FAI');
  
  // Calculate TI
  var ti = red.subtract(green).subtract(nir.subtract(green.multiply(0.5))).rename('TI');
  
  // Add bands and apply water mask
  return image.addBands(fai).addBands(ti).updateMask(waterMaskImage);
}

```
### 3.3 Cloud Masking
Clouds have reflectance characteristics in the visible to near-infrared bands that are similar to cyanobacterial blooms, which can result in false positives with high FAI values. Therefore, cloud removal is necessary.

Cloud masking uses two specialized spectral indices:
$I_1 = \frac{B1 - B7}{B1 + B7}$，A value greater than 0.08 typically indicates thin clouds.
$I_2 = \frac{B5 - B4}{B5 + B7}$，When $I_2$ is less than 0.1 and $I_1$ is relatively high, it helps identify thin cloud features.

First, thick cloud areas are detected using the characteristic that the reflectance of the red band (B4) is greater than 0.2.  
Second, thin clouds and shadows are identified using the aerosol band (B1) and red-edge band 3 (B7) index (I1), and the red-edge band 1 (B5) and red band index (I2), by applying the conditions I1 > 0.08 and I2 < 0.1 to exclude these pixels.  
Finally, the thick and thin cloud detection results are combined to generate a comprehensive cloud mask, retaining only cloud-free pixels for subsequent analysis.
```js
function maskCloud(image) {
  // Normalize reflectance
  var b1 = image.select('B1').divide(10000); // Aerosol band (442 nm)
  var b4 = image.select('B4').divide(10000); // Red band (664 nm)
  var b5 = image.select('B5').divide(10000); // Red-edge 1 band (704 nm)
  var b7 = image.select('B7').divide(10000); // Red-edge 3 band (782 nm)
  
  // Basic cloud mask (B4 > 0.2)
  var cloudMask = b4.lt(0.2);
  
  // Thin cloud/shadow removal
  var I1 = b1.subtract(b7).divide(b1.add(b7)); // (B1-B7)/(B1+B7)
  var I2 = b5.subtract(b4).divide(b5.add(b7)); // (B5-B4)/(B5+B7)
  var thinCloudMask = I1.gt(0.08).and(I2.lt(0.1)).not();
  
  // Combined cloud mask
  return image.updateMask(cloudMask.and(thinCloudMask));
}
```

### 3.4 High-Turbidity Water Mask
High-turbidity water bodies (e.g., those with large amounts of sediment or suspended matter) have optical characteristics similar to cyanobacterial blooms, potentially leading to false positives.  
To eliminate the influence of high-turbidity waters, we apply a mask based on the Turbidity Index (TI).
```js
// Turbidity mask (TI > 0)
  var ti = red.subtract(green).subtract(
    nir.subtract(green).multiply(0.5)
  ).rename('TI');
```
We use TI > 0 as one of the criteria for distinguishing cyanobacterial blooms, helping to filter out turbid waters not caused by algae.
The complete masking process is as follows:
```js
// Set MNDWI threshold to generate a water mask
var waterMask = mndwi.gt(0.1).selfMask(); // MNDWI > 0.1 indicates water

// Convert water mask to vector
var waterVector = waterMask.clip(chaohu).reduceToVectors({
  geometryType: 'polygon',
  reducer: ee.Reducer.countEvery(),
  scale: 30,
  maxPixels: 10e6,
  bestEffort: true
});

// Filter out water bodies smaller than the threshold (in square meters)
var minArea = 1000000; // Keep water bodies larger than 1,000,000 m²
var filteredWater = waterVector.filter(ee.Filter.gte('count', minArea));

// Extract water body boundaries and buffer inward by 20 meters to eliminate edge effects
var bufferedWaterOutline = filteredWater.map(function(feature) {
  return feature.buffer(-20); // Inward buffer of 20 meters
});

// Create a water mask image for subsequent processing
var waterMaskImage = ee.Image().byte().paint({
  featureCollection: filteredWater,
  color: 1
}).clip(chaohu);

```
Through this series of masking operations, we effectively remove clouds, thin clouds, shadows, and highly turbid waters, thereby improving the accuracy of cyanobacterial bloom detection.
Specifically, a pixel is marked as a cyanobacterial bloom if it simultaneously satisfies FAI > -0.004 and TI ≤ 0:
```js
function detectAlgae(image) {
  // Get FAI and TI bands
  var fai = image.select('FAI');
  var ti = image.select('TI');
  
  // Cyanobacterial bloom condition: FAI > -0.004 and TI ≤ 0
  var algaeMask = fai.gt(-0.004).and(ti.lte(0));
  
  return image.addBands(algaeMask.rename('algae'));
}

var s2WithAlgae = s2WithIndices.map(detectAlgae);
```
### 3.5 Temporal and Spatial Analysis (SDFI)

We perform temporal analysis based on daily bloom area calculations.  
The time series charts help us analyze seasonal patterns of blooms (e.g., frequent occurrences in summer) and interannual trends (e.g., effects of management interventions).
```js
// Calculate daily cyanobacterial bloom area
function calculateDailyBloomArea(image) {
  var bloom = image.select('algae');
  var area = bloom.multiply(ee.Image.pixelArea()).rename('bloom_area');
  var date = image.date().format('YYYY-MM-dd');
  
  return ee.Feature(null, {
    'date': date,
    'bloom_area': area,
    'system:time_start': image.get('system:time_start')
  });
}
var dailyStats = s2WithAlgae.map(calculateDailyBloomArea);
```
To quantify the spatiotemporal dynamics of cyanobacterial blooms, we introduce the Spatial Distribution Frequency Index (SDFI) for spatial analysis:
$$
\text{SDFI} = \frac{\sum_{i=1}^{n} R_{i,\text{bloom}}}{\sum_{i=1}^{n} R_{i,\text{bloom}} + \sum_{i=1}^{n} R_{i,\text{water}}} \times 100\%
$$
SDFI measures the frequency of cyanobacterial blooms at each geographic location.
It describes the percentage of observations during the monitoring period in which a pixel was identified as having a bloom, reflecting the bloom-prone nature of the location.
```js
// Calculate SDFI
var bloomCollection = s2WithAlgae.select('algae');
var bloomSum = bloomCollection.sum();
var validCount = bloomCollection.count();
var sdfi = bloomSum.divide(validCount).multiply(100).rename('SDFI');
```
Based on the SDFI results, we can identify high-frequency bloom areas and prioritize them for strengthened management efforts.

### 3.6 Bloom Classification
Based on FAI values and red band reflectance, we established a cyanobacterial bloom classification system that divides blooms into five levels (VCI_Lv2 to VCI_Lv6), quantifying algal aggregation density and ecological risk.
```js
function classifyBloom(image) {
  var fai = image.select('FAI');
  var red = image.select('B4').divide(10000);
  
  // Classification criteria
  var lv2 = fai.gte(-0.004).and(fai.lt(0.1));    
  var lv3 = fai.gte(0.1).and(fai.lt(0.15));      
  var lv4 = fai.gte(0.15).and(fai.lt(0.2));      
  var lv5 = fai.gte(0.2).and(red.lt(0.07));      
  var lv6 = fai.gte(0.2).and(red.gte(0.07));     
  
  return image
    .addBands(lv2.rename('VCI_Lv2'))
    .addBands(lv3.rename('VCI_Lv3'))
    .addBands(lv4.rename('VCI_Lv4'))
    .addBands(lv5.rename('VCI_Lv5'))
    .addBands(lv6.rename('VCI_Lv6'));
}
var classified = s2WithAlgae.map(classifyBloom);
```

### 3.7 Interactive Visualization Module

Below is the full code for the interactive visualization module, implemented in Google Earth Engine. Each functional block is commented for clarity.

1. Create the main UI panel and insert to left

Create a fixed-width panel and insert it into the left side of the interface to hold all UI elements.
```js
// 1. Create the main UI panel and insert to left
var panel = ui.Panel({style: {width: '360px'}});
ui.root.insert(0, panel);
```
2. Title of the application

Add a title label to clearly show that the app monitors algal blooms in Chaohu Lake.
```js
// 2. Title of the application
panel.add(ui.Label({
  value: 'Chaohu Lake Algal Bloom Monitoring',
  style: {fontSize: '22px', fontWeight: 'bold'}
}));
```
3. Lake area calculation tool

Provide a button to calculate and display the total water area of Chaohu Lake based on satellite data.
```js
// 3. Lake area calculation tool
panel.add(ui.Label('Basic Information Tool:', {fontWeight: 'bold', margin: '8px 0 4px 0'}));

var areaLabel = ui.Label('', {color: 'gray', fontStyle: 'italic'});
var lakeAreaButton = ui.Button({
  label: 'Calculate Total Lake Area (km²)',
  style: {stretch: 'horizontal', margin: '6px 0'},
  onClick: function() {
    areaLabel.setValue('Calculating, please wait...');
    var lakeArea = waterMask.multiply(ee.Image.pixelArea())
      .reduceRegion({
        reducer: ee.Reducer.sum(),
        geometry: chaohu,
        scale: 10,
        maxPixels: 1e10
      }).get('MNDWI');
    ee.Number(lakeArea).divide(1e6).evaluate(function(result) {
      areaLabel.setValue('Total lake area is approximately: ' + result.toFixed(2) + ' km²');
    });
  }
});
panel.add(lakeAreaButton);
panel.add(areaLabel);
```
4. Time slider for selecting date

Add a slider so users can select different dates and view corresponding algal bloom conditions.
```js
// 4. Time slider for selecting date
var dateList = s2WithAlgae.aggregate_array('system:time_start').map(function(d) {
  return ee.Date(d).format('YYYY-MM-dd');
}).distinct();
dateList.evaluate(function(list) {
  var slider = ui.DateSlider({
    start: ee.Date(startDate),
    end: ee.Date(endDate),
    period: 1,
    style: {stretch: 'horizontal'},
    onChange: function(range) {
      var selectedDate = ee.Date(range.start());
      updateMap(selectedDate);
    }
  });
  panel.add(ui.Label('Select Date:'));
  panel.add(slider);
});
```
5. Map layer management functions

Define functions to clear and reload key map layers, keeping the interface organized.
```js
// 5. Map layer management functions
function clearMap() {
  Map.layers().reset();
  Map.addLayer(baseImage, {
    bands: ['B4', 'B3', 'B2'],
    min: 0,
    max: 3000
  }, 'Base Image (Cloud Masked)');
  Map.addLayer(bufferedWaterOutline, {
    color: 'blue',
    fillColor: '00000000',
    width: 2
  }, 'Chaohu Lake Outline');
  Map.addLayer(s2WithIndices.select('FAI').mean(), {
    min: -0.1, 
    max: 0.1, 
    palette: ['blue', 'white', 'yellow']
  }, 'FAI Mean');
  Map.addLayer(sdfi, {
    min: 0,
    max: 100,
    palette: ['white', 'green', 'red']
  }, 'SDFI');
}
function updateMap(date) {
  clearMap();
  var image = s2WithAlgae.filterDate(date, date.advance(1, 'day')).first();
  var algae = image.select('algae');
  Map.addLayer(algae, {min: 0, max: 1, palette: ['white', 'red']}, 'Algae Mask (' + date.format('YYYY-MM-dd').getInfo() + ')');
}
clearMap();
```
6. Add legend for overlays

Add a legend explaining the colors used on the map for easier interpretation of bloom and water features.
```js
// 6. Add legend for overlays
var legend = ui.Panel({style: {position: 'bottom-left', padding: '8px 15px'}});
legend.add(ui.Label({value: 'Legend', style: {fontWeight: 'bold'}}));
function makeLegendRow(color, name) {
  var colorBox = ui.Label({style: {backgroundColor: color, padding: '8px'}});
  var description = ui.Label({value: name, style: {margin: '0 0 4px 6px'}});
  return ui.Panel([colorBox, description], ui.Panel.Layout.Flow('horizontal'));
}
legend.add(makeLegendRow('red', 'Algae Mask'));
legend.add(makeLegendRow('yellow', 'FAI Mean'));
legend.add(makeLegendRow('green', 'SDFI'));
legend.add(makeLegendRow('blue', 'Lake Outline'));
Map.add(legend);
```
7. Toggle VCI intensity layers

Allow users to turn different bloom intensity layers on or off for customized visualization.
```js
// 7. Toggle VCI intensity layers
panel.add(ui.Label('Select Algal Bloom Intensity Layers:', {fontWeight: 'bold'}));
var vciLayers = {
  'VCI_Lv2': {image: vciMean.select('VCI_Lv2'), vis: vciPalette['VCI_Lv2']},
  'VCI_Lv3': {image: vciMean.select('VCI_Lv3'), vis: vciPalette['VCI_Lv3']},
  'VCI_Lv4': {image: vciMean.select('VCI_Lv4'), vis: vciPalette['VCI_Lv4']},
  'VCI_Lv5': {image: vciMean.select('VCI_Lv5'), vis: vciPalette['VCI_Lv5']},
  'VCI_Lv6': {image: vciMean.select('VCI_Lv6'), vis: vciPalette['VCI_Lv6']}
};
var activeLayers = {};
Object.keys(vciLayers).forEach(function(layerName) {
  var checkbox = ui.Checkbox({
    label: layerName,
    value: false,
    onChange: function(checked) {
      if (checked) {
        var lyr = Map.addLayer(vciLayers[layerName].image, vciLayers[layerName].vis, layerName);
        activeLayers[layerName] = lyr;
      } else {
        Map.layers().remove(activeLayers[layerName]);
      }
    }
  });
  panel.add(checkbox);
});
```
8. Button to reset map layers

Add a reset button that quickly clears all active layers for a fresh start.
```js
// 8. Button to reset map layers
var resetButton = ui.Button({
  label: 'Reset Map Layers',
  style: {stretch: 'horizontal', margin: '6px 0'},
  onClick: function() {
    clearMap();
  }
});
panel.add(resetButton);
```
9. Algae area chart generation

Provide a button to generate a daily bloom area chart, helping track bloom trends over time.
```js
// 9. Algae area chart generation
panel.add(ui.Label('Analysis Tools:', {fontWeight: 'bold', margin: '12px 0 4px 0'}));
var chartStatus = ui.Label('', {color: 'gray', fontStyle: 'italic'});
var chartContainer = ui.Panel();
var generateChartButton = ui.Button({
  label: 'Generate Daily Algae Area Chart',
  style: {stretch: 'horizontal', margin: '8px 0'},
  onClick: function() {
    chartStatus.setValue('Generating chart, please wait...');
    chartContainer.clear();
    var chart = ui.Chart.feature.byFeature(dailyStats, 'date', 'bloom_area_km2')
      .setChartType('LineChart')
      .setOptions({
        title: 'Daily Algal Bloom Area (km²)',
        hAxis: {title: 'Date'},
        vAxis: {title: 'Area (km²)'},
        lineWidth: 2,
        pointSize: 3,
        colors: ['#1f78b4']
      });
    chartContainer.add(chart);
    chartStatus.setValue('Chart generated');
  }
});
panel.add(generateChartButton);
panel.add(chartStatus);
panel.add(chartContainer);
```
10. Export data buttons

Add buttons to export daily, monthly, or yearly bloom data for further offline analysis.
```js
// 10. Export data buttons
panel.add(ui.Label('Data Export Tools:', {fontWeight: 'bold', margin: '12px 0 4px 0'}));
var exportStatus = ui.Label('', {color: 'gray', fontStyle: 'italic'});
panel.add(exportStatus);

panel.add(ui.Button({
  label: 'Export Daily Data',
  onClick: function() {
    exportStatus.setValue('Exporting daily data. Please click "Run" in the Tasks tab...');
    Export.table.toDrive({
      collection: dailyStats,
      description: 'Daily_Chaohu_Bloom_Stats',
      fileFormat: 'CSV',
      selectors: ['date', 'year', 'month', 'bloom_area_km2', 'ratio_percent']
    });
  }
}));
panel.add(ui.Button({
  label: 'Export Monthly Data',
  onClick: function() {
    exportStatus.setValue('Exporting monthly data. Please click "Run" in the Tasks tab...');
    Export.table.toDrive({
      collection: monthlyStats,
      description: 'Monthly_Chaohu_Bloom_Stats',
      fileFormat: 'CSV',
      selectors: ['year_month', 'month', 'monthly_bloom_area_km2', 'monthly_ratio_percent', 'observation_count']
    });
  }
}));
panel.add(ui.Button({
  label: 'Export Yearly Data',
  onClick: function() {
    exportStatus.setValue('Exporting yearly data. Please click "Run" in the Tasks tab...');
    Export.table.toDrive({
      collection: yearlyStats,
      description: 'Yearly_Chaohu_Bloom_Stats',
      fileFormat: 'CSV',
      selectors: ['year', 'yearly_bloom_area_km2', 'yearly_ratio_percent', 'observation_count']
    });
  }
}));
```


### References
[1]Brooks, B.W., Lazorchak, J.M., Howard, M.D., Johnson, M.V.V., Morton, S.L., Perkins, D.A., Reavie, E.D., Scott, G.I., Smith, S.A. and Steevens, J.A., 2016. Are harmful algal blooms becoming the greatest inland water quality threat to public health and aquatic ecosystems?. Environmental toxicology and chemistry, 35(1), pp.6-13.

[2]Jing, Y., Zhang, Y., Hu, M., Chu, Q. and Ma, R., 2019. MODIS-satellite-based analysis of long-term temporal-spatial dynamics and drivers of algal blooms in a plateau lake Dianchi, China. Remote Sensing, 11(21), p.2582.

[3]Guo, H., Liu, H., Lyu, H., Bian, Y., Zhong, S., Li, Y., Miao, S., Yang, Z., Xu, J., Cao, J. and Li, Y., 2022. Is there any difference on cyanobacterial blooms patterns between Lake Chaohu and Lake Taihu over the last 20 years?. Environmental Science and Pollution Research, 29(27), pp.40941-40953.

[4]Song, T., Xu, C., Yan, F. & Zhang, J. (2025) 'Spatiotemporal Monitoring of Lake Cyanobacterial Blooms Based on Sentinel-2 Data', China Environmental Monitoring, 41(1), pp. 214–224
[5]Hu, C., 2009. A novel ocean color index to detect floating algae in the global oceans. Remote Sensing of Environment, 113(10), pp.2118-2129.
